In [3]:
import kagglehub
import pandas as pd
import os
import pandas as pd
import numpy as np

In [4]:
# Download the dataset from kaggle
ds_movies = kagglehub.dataset_download("carolzhangdc/imdb-5000-movie-dataset")
df_movies = pd.read_csv(os.path.join(ds_movies, "movie_metadata.csv"))

# Save to local repository
local_path = "../data/raw/imdb_5000_movies.csv"
os.makedirs(os.path.dirname(local_path), exist_ok=True)
df_movies.to_csv(local_path, index=False)

print(f"Dataset saved to: {local_path}")

Dataset saved to: data/raw/imdb_5000_movies.csv


In [5]:
# Load movies from corpus
movies = pd.read_csv('../data/MovieSummaries/movie.metadata.tsv', sep='\t', header=None)
movies.columns = ['wikipedia_movie_id', 'freebase_movie_id', 'movie_name', 'movie_release_date', 'movie_box_office_revenue', 'movie_runtime', 'movie_languages', 'movie_countries', 'movie_genres']

In [7]:
# Load IMDB
imdb_movies = pd.read_csv('../data/raw/imdb_5000_movies.csv')
imdb_movies['movie_title'] = imdb_movies['movie_title'].str.strip().str.replace(u'\xa0', '')
imdb_movies = imdb_movies[['movie_title', 'gross']]

# Convert 'gross' to numeric pour fill les empty values après
imdb_movies['gross'] = pd.to_numeric(imdb_movies['gross'], errors='coerce')
imdb_movies = imdb_movies.dropna(subset=['gross'])

In [10]:
# Convert 'movie_box_office_revenue' to numeric, handling missing values
movies['movie_box_office_revenue'] = pd.to_numeric(movies['movie_box_office_revenue'], errors='coerce')

# Merge the two DataFrames on movie name/title
merged_movies = pd.merge(
    movies, imdb_movies,
    left_on='movie_name', right_on='movie_title',
    how='left'
)

# Update 'movie_box_office_revenue' where it's NaN with 'gross' from IMDB
merged_movies['movie_box_office_revenue'] = merged_movies['movie_box_office_revenue'].fillna(merged_movies['gross'])
merged_movies.drop(columns=['movie_title', 'gross'], inplace=True)
merged_movies = merged_movies.dropna(subset=['movie_box_office_revenue'])